In [1]:
!pip install opensoundscape==0.6.2
from opensoundscape.preprocess.preprocessors import BasePreprocessor
import torch
import pandas as pd
from pathlib import Path
import numpy as np
import pandas as pd
from opensoundscape.torch.models import cnn
from opensoundscape.torch.architectures import cnn_architectures
from WTS_chunking import annotation_chunker_no_duplicates, extact_split, fill_no_class


Bad key "text.kerning_factor" on line 4 in
C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
#COSMOS_BirdNET-Lite_Labels_100.csv
name = "automated_cosmos_tweety_to_file"
main_name ="C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/automated_cosmos_tweety_to_file.csv"#"./COSMOS_Microfaune-Filename_Labels_100.csv"#"./COSMOS_BirdNET-Lite-Filename_Labels_05Conf.csv" #'./COSMOS_BirdNET-Lite_Labels_05Conf.csv' #Tweety birdnet, #tweetyfilename
df = pd.read_csv(main_name, sep=',')
#warblr = pd.read_csv("./warblrb10k_public_metadata_2018.csv")
#no_bird = warblr[warblr["hasbird"] == 0].reset_index(drop=True)
#no_bird["OFFSET"] = float(0)
#no_bird["DURATION"] = float(10)
#no_bird["FOLDER"] = "C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/wav/"
#no_bird["CHANNEL"] = 0
#no_bird["CLIP LENGTH"] = float(10)
#no_bird["MANUAL ID"] = "no bird"
#no_bird["CONFIDENCE"] = 1
#no_bird["SAMPLE RATE"] = 44100
#no_bird["FILENAME"] = no_bird["itemid"]+".wav"
#no_bird["IN FILE"] = no_bird["FILENAME"]
no_bird = pd.read_csv("C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/ICML_NO_BIRD_SAMPLE.csv")
df = df.append(no_bird)


test_name = "./" +name + "_TESTING.csv" #"./COSMOS_BirdNET-Lite_Labels_100_TESTING.csv"
train_name ="./" +name + "_TRAINING.csv" #"./COSMOS_BirdNET-Lite_Labels_100_TRAINING.csv"
THIS_IS_NEW_DATA = True
#df['IN FILE'] = [f.split('/')[-1] for f in df['FILENAME']]
print(df["IN FILE"])

0        XC100027 - Southern Nightingale-Wren - Microce...
1        XC100027 - Southern Nightingale-Wren - Microce...
2        XC100027 - Southern Nightingale-Wren - Microce...
3        XC100027 - Southern Nightingale-Wren - Microce...
4        XC100027 - Southern Nightingale-Wren - Microce...
                               ...                        
30934                      Zimmerius-gracilipes-258669.wav
30935                      Zimmerius-gracilipes-258669.wav
30936                      Zimmerius-gracilipes-258669.wav
30937                      Zimmerius-gracilipes-258669.wav
30938                      Zimmerius-gracilipes-258669.wav
Name: IN FILE, Length: 86586, dtype: object


In [3]:
no_bird

,Unnamed: 0,Unnamed: 0.1,IN FILE,CLIP LENGTH,OFFSET,DURATION,MAX FREQ,MIN FREQ,SAMPLE RATE,MANUAL ID,TIME_SPENT,LAST MOD BY,LAST MOD DATE,LAST MOD TIME,FOLDER,FILE_PATH,FILENAME,CHANNEL,CONFIDENCE
0,0,0,Aburria-aburri-257423.wav,24.6335,0.0,24.6335,24000,0,44100,other,1050.630,alarce,11/22/2021,2:13:53,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,NaN,Aburria-aburri-257423.wav,0,1
1,1,1,Aburria-aburri-257423.wav,24.6335,0.0,24.6335,24000,0,44100,other,1050.630,alarce,11/22/2021,2:13:53,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,NaN,Aburria-aburri-257423.wav,0,1
2,2,2,Aburria-aburri-257423.wav,24.6335,0.0,24.6335,24000,0,44100,other,1050.630,alarce,11/22/2021,2:13:53,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,NaN,Aburria-aburri-257423.wav,0,1
3,3,3,Aburria-aburri-387212.wav,84.4800,0.0,84.4800,24000,0,44100,other,184.388,alarce,11/22/2021,3:28:01,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,NaN,Aburria-aburri-387212.wav,0,1
4,4,4,Aburria-aburri-387212.wav,84.4800,0.0,84.4800,24000,0,44100,other,184.385,alarce,11/22/2021,3:28:01,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,NaN,Aburria-aburri-387212.wav,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30934,30934,31336,Zimmerius-gracilipes-258669.wav,27.5592,0.0,27.5592,24000,0,44100,other,217.078,alarce,10/18/2021,18:32:32,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,DELETE,Zimmerius-gracilipes-258669.wav,0,1
30935,30935,31337,Zimmerius-gracilipes-258669.wav,27.5592,0.0,27.5592,24000,0,44100,other,217.080,alarce,10/18/2021,18:32:32,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,DELETE,Zimmerius-gracilipes-258669.wav,0,1
30936,30936,31338,Zimmerius-gracilipes-258669.wav,27.5592,0.0,27.5592,24000,0,44100,other,217.079,alarce,10/18/2021,18:32:32,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,DELETE,Zimmerius-gracilipes-258669.wav,0,1
30937,30937,31339,Zimmerius-gracilipes-258669.wav,27.5592,0.0,27.5592,24000,0,44100,other,217.081,alarce,10/18/2021,18:32:32,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,DELETE,Zimmerius-gracilipes-258669.wav,0,1


In [4]:
ef = pd.read_csv('C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/cosmos_labeled_data_files_added.csv')
ef = annotation_chunker_no_duplicates(ef, 3, True)
ef['IN FILE'] = ef['IN FILE'].apply(lambda x : x.replace("_", " "))
gtClips = np.array(ef['IN FILE'].unique())
ef.shape

(6621, 9)

In [5]:
ef["IN FILE"][0]

'XC87449 - Scaly-breasted Woodpecker - Celeus grammicus subcervinus.mp3'

In [6]:

j = 0
ff = ef#pd.DataFrame(columns=df.columns)
#for i, row in ef.iterrows():
#    if(row['IN FILE'] in gtClips):
#        # print(row['IN FILE'])
##        ff.loc[j] = row
#        df.drop(i, inplace=True)
#        j += 1
#print(df.shape)
try:
    df = df[df['CONFIDENCE'] >= 0.8]
except:
    pass

In [7]:
ff

,FOLDER,IN FILE,CLIP LENGTH,CHANNEL,OFFSET,DURATION,SAMPLE RATE,MANUAL ID,CONFIDENCE
0,./COSMOS,XC87449 - Scaly-breasted Woodpecker - Celeus g...,23.2956,0,0.0,3.0,44100,Celeus grammicus,0.0
1,./COSMOS,XC87449 - Scaly-breasted Woodpecker - Celeus g...,23.2956,0,3.0,3.0,44100,Celeus grammicus,0.0
2,./COSMOS,XC87449 - Scaly-breasted Woodpecker - Celeus g...,23.2956,0,6.0,3.0,44100,no bird,0.0
3,./COSMOS,XC87449 - Scaly-breasted Woodpecker - Celeus g...,23.2956,0,9.0,3.0,44100,Celeus grammicus,0.0
4,./COSMOS,XC87449 - Scaly-breasted Woodpecker - Celeus g...,23.2956,0,12.0,3.0,44100,Celeus grammicus,0.0
...,...,...,...,...,...,...,...,...,...
6616,./COSMOS,XC27942 - Yellow-olive Flatbill - Tolmomyias s...,31.3404,0,15.0,3.0,44100,Tolmomyias sulphurescens,0.0
6617,./COSMOS,XC27942 - Yellow-olive Flatbill - Tolmomyias s...,31.3404,0,18.0,3.0,44100,no bird,0.0
6618,./COSMOS,XC27942 - Yellow-olive Flatbill - Tolmomyias s...,31.3404,0,21.0,3.0,44100,Tolmomyias sulphurescens,0.0
6619,./COSMOS,XC27942 - Yellow-olive Flatbill - Tolmomyias s...,31.3404,0,24.0,3.0,44100,Tolmomyias sulphurescens,0.0


In [8]:
len(df['IN FILE'].unique())

5126

In [9]:
df['IN FILE'] = ['./Cosmos_data/Training/' + f for f in df['IN FILE']]
ff['IN FILE CURRENT'] = ['./Cosmos_data/Training_Xeno_Canto_2022/' + f for f in ff['IN FILE']]
ff['IN FILE'] = ['./Cosmos_data/Testing_Xeno_Canto_2022/' + f for f in ff['IN FILE']]


In [10]:
ef = ff.groupby('IN FILE', group_keys=False).apply(lambda ff: ff.sample(1))
ef['IN FILE'] = [f.replace(".mp3", ".wav") for f in ef['IN FILE']]
import os
from pydub import AudioSegment

path = "C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/"

#Change working directory
os.chdir(path)

audio_files = os.listdir()

# You dont need the number of files in the folder, just iterate over them directly using:
for file in audio_files:
    #spliting the file into the name and the extension
    name, ext = os.path.splitext(file)
    if ext == ".mp3":
        try:
           mp3_sound = AudioSegment.from_mp3(file)
           #rename them using the old name + ".wav"
           mp3_sound.export("{0}.wav".format(name), format="wav")
        except:
            continue
ef

,FOLDER,IN FILE,CLIP LENGTH,CHANNEL,OFFSET,DURATION,SAMPLE RATE,MANUAL ID,CONFIDENCE,IN FILE CURRENT
3415,./COSMOS,./Cosmos_data/Testing_Xeno_Canto_2022/XC100864...,9.39575,0,3.0,3.0,44100,bird,0.0,./Cosmos_data/Training_Xeno_Canto_2022/XC10086...
3300,./COSMOS,./Cosmos_data/Testing_Xeno_Canto_2022/XC101469...,137.41000,0,15.0,3.0,44100,Turdus leucomelas,0.0,./Cosmos_data/Training_Xeno_Canto_2022/XC10146...
3419,./COSMOS,./Cosmos_data/Testing_Xeno_Canto_2022/XC101564...,63.97150,0,6.0,3.0,44100,no bird,0.0,./Cosmos_data/Training_Xeno_Canto_2022/XC10156...
4034,./COSMOS,./Cosmos_data/Testing_Xeno_Canto_2022/XC101694...,23.05110,0,6.0,3.0,44100,no bird,0.0,./Cosmos_data/Training_Xeno_Canto_2022/XC10169...
2682,./COSMOS,./Cosmos_data/Testing_Xeno_Canto_2022/XC101816...,34.43860,0,21.0,3.0,44100,Turdus leucomelas,0.0,./Cosmos_data/Training_Xeno_Canto_2022/XC10181...
...,...,...,...,...,...,...,...,...,...,...
6017,./COSMOS,./Cosmos_data/Testing_Xeno_Canto_2022/XC96443 ...,32.29380,0,15.0,3.0,44100,bird,0.0,./Cosmos_data/Training_Xeno_Canto_2022/XC96443...
4640,./COSMOS,./Cosmos_data/Testing_Xeno_Canto_2022/XC96647 ...,108.87900,0,54.0,3.0,48000,no bird,0.0,./Cosmos_data/Training_Xeno_Canto_2022/XC96647...
1359,./COSMOS,./Cosmos_data/Testing_Xeno_Canto_2022/XC97532 ...,20.33650,0,3.0,3.0,44100,Bird,0.0,./Cosmos_data/Training_Xeno_Canto_2022/XC97532...
4572,./COSMOS,./Cosmos_data/Testing_Xeno_Canto_2022/XC97542 ...,4.40037,0,0.0,3.0,44100,Myioborus miniatus,0.0,./Cosmos_data/Training_Xeno_Canto_2022/XC97542...


In [11]:
#ef["IN FILE"][2]

In [12]:
import shutil

def Move_Files(filename):
    file = filename.split("/")
    og_file_path = "C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/" + file[3]
    new_file_path = "C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/" + "/".join(file[2:])
    try:
        if(THIS_IS_NEW_DATA):
            shutil.copyfile(og_file_path, new_file_path)
    except:
        return "DELETE"
    return new_file_path
#Move_Files('./Cosmos_data/Testing_Xeno_Canto_2022/XC100864 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis.mp3')
#shutil.copyfile('Cosmos_data/XC100864 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis.mp3', './Cosmos_data/Testing_Xeno_Canto_2022/XC100864 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis.mp3')
#shutil.copyfile("/home/shperry/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/XC100864 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis.mp3", '/home/shperry/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Testing_Xeno_Canto_2022/XC100864 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis.mp3')
ef["IN FILE"] = ef["IN FILE"].apply(Move_Files)


In [13]:
ef = ef[ef["IN FILE"] != "DELETE"]

In [14]:
ef["IN FILE"].iloc[0]

'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Testing_Xeno_Canto_2022/XC100864 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis.wav'

In [15]:
"C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/XC104217 - Slate-throated Whitestart - Myioborus miniatus.mp3"


'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/XC104217 - Slate-throated Whitestart - Myioborus miniatus.mp3'

In [16]:
'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/XC104217 - Slate-throated Whitestart - Myioborus miniatus.wav'

'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/XC104217 - Slate-throated Whitestart - Myioborus miniatus.wav'

In [17]:
from opensoundscape.audio import Audio
import os
clip_duration = 3
clip_overlap = 0
final_clip = None
clip_dir = 'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Testing_Cosmos'
# classes = labels
min_label_overlap = 0.1

ef_split_save = pd.DataFrame(columns=['file', 'start_time', 'end_time', 'SAMPLING RATE', 'MANUAL ID'])

cnt = 0
#ef = ef.reset_index()
print(ef)
for i, row in ef.iterrows():
    try:
        audio = Audio.from_file(row['IN FILE'])
    except Exception as e:
        print(e)
        print(row['IN FILE'] + ' not found')
        continue
    file_name = row['IN FILE'].split('/')
    # print(file_name)
    clip_df = audio.split_and_save(
        clip_dir,
        prefix=file_name[-1],
        clip_duration=clip_duration,
        clip_overlap=clip_overlap,
        final_clip='remainder',
        dry_run=(os.path.exists(clip_dir + file_name[-1]))
    )
    clip_df['SAMPLING RATE'] = [44100] * clip_df.shape[0]
    clip_df['MANUAL ID'] = [row['MANUAL ID']] * clip_df.shape[0]
    clip_df.reset_index(inplace=True)
    ef_split_save = ef_split_save.append(clip_df, ignore_index=True)
    print(i, "out of", ef.shape[0])

print(ef_split_save)
ef_split_save.to_csv(test_name)

        FOLDER                                            IN FILE  \
3415  ./COSMOS  C:/Users/siloux/Desktop/E4E/passive-acoustic-b...   
3300  ./COSMOS  C:/Users/siloux/Desktop/E4E/passive-acoustic-b...   
3419  ./COSMOS  C:/Users/siloux/Desktop/E4E/passive-acoustic-b...   
4034  ./COSMOS  C:/Users/siloux/Desktop/E4E/passive-acoustic-b...   
2682  ./COSMOS  C:/Users/siloux/Desktop/E4E/passive-acoustic-b...   
...        ...                                                ...   
6017  ./COSMOS  C:/Users/siloux/Desktop/E4E/passive-acoustic-b...   
4640  ./COSMOS  C:/Users/siloux/Desktop/E4E/passive-acoustic-b...   
1359  ./COSMOS  C:/Users/siloux/Desktop/E4E/passive-acoustic-b...   
4572  ./COSMOS  C:/Users/siloux/Desktop/E4E/passive-acoustic-b...   
3342  ./COSMOS  C:/Users/siloux/Desktop/E4E/passive-acoustic-b...   

      CLIP LENGTH  CHANNEL  OFFSET  DURATION  SAMPLE RATE  \
3415      9.39575        0     3.0       3.0        44100   
3300    137.41000        0    15.0       3.0     

913 out of 483
931 out of 483
6487 out of 483
3787 out of 483
4299 out of 483
2233 out of 483
553 out of 483
3795 out of 483
1017 out of 483
3051 out of 483
1031 out of 483
1053 out of 483
1057 out of 483
1105 out of 483
5382 out of 483
1854 out of 483
3832 out of 483
5043 out of 483
5088 out of 483
4513 out of 483
2480 out of 483
2526 out of 483
2245 out of 483
4314 out of 483
2246 out of 483
3061 out of 483
3088 out of 483
1858 out of 483
3365 out of 483
3996 out of 483
4571 out of 483
3997 out of 483
1144 out of 483
6596 out of 483
6500 out of 483
3106 out of 483
1866 out of 483
3873 out of 483
575 out of 483
6505 out of 483
4327 out of 483
6523 out of 483
3123 out of 483
1928 out of 483
3894 out of 483
3251 out of 483
6538 out of 483
6546 out of 483
1164 out of 483
3372 out of 483
5391 out of 483
5394 out of 483
5398 out of 483
1877 out of 483
5098 out of 483
2248 out of 483
3904 out of 483
2044 out of 483
1130 out of 483
4334 out of 483
5962 out of 483
576 out of 483
3135 out of 4

In [18]:
df

C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,OFFSET,DURATION,FOLDER,IN FILE,CHANNEL,CLIP LENGTH,SAMPLE RATE,MANUAL ID,CONFIDENCE,FILEPATH,Unnamed: 0.1,MAX FREQ,MIN FREQ,TIME_SPENT,LAST MOD BY,LAST MOD DATE,LAST MOD TIME,FILE_PATH,FILENAME
10,10,24.906977,1.093023,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,./Cosmos_data/Training/XC100027 - Southern Nig...,0,81.528163,44100,Microcerculus marginatus,0.823617,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,14,30.232558,0.697674,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,./Cosmos_data/Training/XC100027 - Southern Nig...,0,81.528163,44100,Microcerculus marginatus,0.812980,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,21,50.627907,1.372093,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,./Cosmos_data/Training/XC100027 - Southern Nig...,0,81.528163,44100,Microcerculus marginatus,0.955846,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,22,60.488372,1.511628,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,./Cosmos_data/Training/XC100027 - Southern Nig...,0,81.528163,44100,Microcerculus marginatus,1.000000,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,29,0.860465,0.465116,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,./Cosmos_data/Training/XC100068 - Yellow-olive...,0,55.092245,44100,Tolmomyias sulphurescens,1.000000,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30934,30934,0.000000,27.559200,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,./Cosmos_data/Training/Zimmerius-gracilipes-25...,0,27.559200,44100,other,1.000000,NaN,31336.0,24000.0,0.0,217.078,alarce,10/18/2021,18:32:32,DELETE,Zimmerius-gracilipes-258669.wav
30935,30935,0.000000,27.559200,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,./Cosmos_data/Training/Zimmerius-gracilipes-25...,0,27.559200,44100,other,1.000000,NaN,31337.0,24000.0,0.0,217.080,alarce,10/18/2021,18:32:32,DELETE,Zimmerius-gracilipes-258669.wav
30936,30936,0.000000,27.559200,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,./Cosmos_data/Training/Zimmerius-gracilipes-25...,0,27.559200,44100,other,1.000000,NaN,31338.0,24000.0,0.0,217.079,alarce,10/18/2021,18:32:32,DELETE,Zimmerius-gracilipes-258669.wav
30937,30937,0.000000,27.559200,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,./Cosmos_data/Training/Zimmerius-gracilipes-25...,0,27.559200,44100,other,1.000000,NaN,31339.0,24000.0,0.0,217.081,alarce,10/18/2021,18:32:32,DELETE,Zimmerius-gracilipes-258669.wav


In [19]:
no_bird["IN FILE"]

C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0              Aburria-aburri-257423.wav
1              Aburria-aburri-257423.wav
2              Aburria-aburri-257423.wav
3              Aburria-aburri-387212.wav
4              Aburria-aburri-387212.wav
                      ...               
30934    Zimmerius-gracilipes-258669.wav
30935    Zimmerius-gracilipes-258669.wav
30936    Zimmerius-gracilipes-258669.wav
30937    Zimmerius-gracilipes-258669.wav
30938    Zimmerius-gracilipes-258669.wav
Name: IN FILE, Length: 30939, dtype: object

In [20]:
import shutil

def Move_Files(filename):
    file = filename.split("/")
    og_file_path = "C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/" + file[3]
    new_file_path = "C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/" + "/".join(file[2:])
    #print(new_file_path)
    name, ext = os.path.splitext(og_file_path)
    if ext == ".mp3":
        try:
           mp3_sound = AudioSegment.from_mp3(og_file_path)
           #rename them using the old name + ".wav"
           mp3_sound.export("{0}.wav".format(name), format="wav")
        except:
            return "DELETE"
    
    if(not os.path.exists(new_file_path)):
        try:
            shutil.copyfile(og_file_path, new_file_path)
        except:
            try:
                mp3_sound = AudioSegment.from_mp3(og_file_path.replace(".wav", ".mp3"))
                #rename them using the old name + ".wav"
                mp3_sound.export("{0}.wav".format(name), format="wav")
                shutil.copyfile(og_file_path, new_file_path)
            except:
                return "DELETE" 
    return new_file_path

def Move_Files_Custom(filename):
    file = filename.split("/")
    og_file_path = "C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/" + file[-1]
    new_file_path = filename
    try:
        if(not os.path.exists(new_file_path)):
            shutil.copyfile(og_file_path, new_file_path)
    except Exception as e:
        print(e)
        return "DELETE"
    return new_file_path


#df = pd.read_csv(main_name, sep=',')
df['IN FILE'] = [f.split('/')[-1].replace(".mp3", ".wav") for f in df['IN FILE']]
df['IN FILE'] = ['./Cosmos_Data/Training_Xeno_Canto_2022/' + f for f in df['IN FILE']]
df["IN FILE"] = df["IN FILE"].apply(Move_Files)
df = df[df["IN FILE"] != "DELETE"]
#warblr = pd.read_csv("C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/warblrb10k_public_metadata_2018.csv")
#no_bird = warblr[warblr["hasbird"] == 0].reset_index(drop=True)
#no_bird["OFFSET"] = float(0)
#no_bird["DURATION"] = float(10)
#no_bird["FOLDER"] = "C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/"
#no_bird["CHANNEL"] = 0
#no_bird["CLIP LENGTH"] = float(10)
#no_bird["MANUAL ID"] = "no bird"
#no_bird["CONFIDENCE"] = 1
#no_bird["SAMPLE RATE"] = 44100
#no_bird["FILENAME"] = no_bird["itemid"]+".wav"
#no_bird["IN FILE"] = 'Training_Xeno_Canto_2022/' + no_bird["FILENAME"]
(no_bird["FOLDER"] + no_bird["IN FILE"]).apply(Move_Files_Custom)
df = df.append(no_bird)
df

[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Accipiter-poliogaster-121768.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Accipiter-poliogaster-121768.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Accipiter-poliogaster-121768.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Accipiter-poliogaster-121768.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Accipiter-poliogaster-121768.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape

[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Aeronautes-montivagus-114387.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Agamia-agami-75995.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Agamia-agami-75995.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Agamia-agami-75995.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Agamia-agami-75995.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Agamia

[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Anhima-cornuta-40357.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Anhima-cornuta-40357.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Anhima-cornuta-40357.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Anhima-cornuta-40357.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Anhima-cornuta-40357.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Anhima

[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Automolus-rufipileatus-47553.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Automolus-rufipileatus-47553.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Automolus-rufipileatus-47553.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Automolus-rufipileatus-47553.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Automolus-rufipileatus-47553.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape

[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Calidris-bairdii-74680.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Calidris-bairdii-74680.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Campephilus-haematogaster-220113.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Campylorhynchus-turdinus-20685.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Campylorhynchus-turdinus-20685.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/Mul

...

[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Tolmomyias-poliocephalus-65590.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Tolmomyias-poliocephalus-65590.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Tolmomyias-poliocephalus-65590.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Tolmomyias-poliocephalus-65590.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Tolmomyias-poliocephalus-65590.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/Open

[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Turdus-ignobilis-446770.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Turdus-ignobilis-446770.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Turdus-ignobilis-446770.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Turdus-ignobilis-446770.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Turdus-ignobilis-446770.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mi

[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Xenops-rutilans-522420.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Xenops-rutilans-522420.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Xenops-rutilans-522420.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Xenops-rutilans-522420.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Xenops-rutilans-522420.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_B

[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Zenaida-auriculata-77090.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Zenaida-auriculata-77090.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Zenaida-auriculata-77090.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Zenaida-auriculata-77090.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Zenaida-auriculata-77090.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifi

,Unnamed: 0,OFFSET,DURATION,FOLDER,IN FILE,CHANNEL,CLIP LENGTH,SAMPLE RATE,MANUAL ID,CONFIDENCE,FILEPATH,Unnamed: 0.1,MAX FREQ,MIN FREQ,TIME_SPENT,LAST MOD BY,LAST MOD DATE,LAST MOD TIME,FILE_PATH,FILENAME
10,10,24.906977,1.093023,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,0.823617,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,14,30.232558,0.697674,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,0.812980,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,21,50.627907,1.372093,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,0.955846,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,22,60.488372,1.511628,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,1.000000,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,29,0.860465,0.465116,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,55.092245,44100,Tolmomyias sulphurescens,1.000000,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30934,30934,0.000000,27.559200,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,Zimmerius-gracilipes-258669.wav,0,27.559200,44100,other,1.000000,NaN,31336.0,24000.0,0.0,217.078,alarce,10/18/2021,18:32:32,DELETE,Zimmerius-gracilipes-258669.wav
30935,30935,0.000000,27.559200,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,Zimmerius-gracilipes-258669.wav,0,27.559200,44100,other,1.000000,NaN,31337.0,24000.0,0.0,217.080,alarce,10/18/2021,18:32:32,DELETE,Zimmerius-gracilipes-258669.wav
30936,30936,0.000000,27.559200,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,Zimmerius-gracilipes-258669.wav,0,27.559200,44100,other,1.000000,NaN,31338.0,24000.0,0.0,217.079,alarce,10/18/2021,18:32:32,DELETE,Zimmerius-gracilipes-258669.wav
30937,30937,0.000000,27.559200,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,Zimmerius-gracilipes-258669.wav,0,27.559200,44100,other,1.000000,NaN,31339.0,24000.0,0.0,217.081,alarce,10/18/2021,18:32:32,DELETE,Zimmerius-gracilipes-258669.wav


In [21]:
df

C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,OFFSET,DURATION,FOLDER,IN FILE,CHANNEL,CLIP LENGTH,SAMPLE RATE,MANUAL ID,CONFIDENCE,FILEPATH,Unnamed: 0.1,MAX FREQ,MIN FREQ,TIME_SPENT,LAST MOD BY,LAST MOD DATE,LAST MOD TIME,FILE_PATH,FILENAME
10,10,24.906977,1.093023,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,0.823617,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,14,30.232558,0.697674,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,0.812980,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,21,50.627907,1.372093,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,0.955846,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,22,60.488372,1.511628,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,1.000000,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,29,0.860465,0.465116,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,55.092245,44100,Tolmomyias sulphurescens,1.000000,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30934,30934,0.000000,27.559200,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,Zimmerius-gracilipes-258669.wav,0,27.559200,44100,other,1.000000,NaN,31336.0,24000.0,0.0,217.078,alarce,10/18/2021,18:32:32,DELETE,Zimmerius-gracilipes-258669.wav
30935,30935,0.000000,27.559200,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,Zimmerius-gracilipes-258669.wav,0,27.559200,44100,other,1.000000,NaN,31337.0,24000.0,0.0,217.080,alarce,10/18/2021,18:32:32,DELETE,Zimmerius-gracilipes-258669.wav
30936,30936,0.000000,27.559200,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,Zimmerius-gracilipes-258669.wav,0,27.559200,44100,other,1.000000,NaN,31338.0,24000.0,0.0,217.079,alarce,10/18/2021,18:32:32,DELETE,Zimmerius-gracilipes-258669.wav
30937,30937,0.000000,27.559200,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,Zimmerius-gracilipes-258669.wav,0,27.559200,44100,other,1.000000,NaN,31339.0,24000.0,0.0,217.081,alarce,10/18/2021,18:32:32,DELETE,Zimmerius-gracilipes-258669.wav


In [22]:
df

C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,OFFSET,DURATION,FOLDER,IN FILE,CHANNEL,CLIP LENGTH,SAMPLE RATE,MANUAL ID,CONFIDENCE,FILEPATH,Unnamed: 0.1,MAX FREQ,MIN FREQ,TIME_SPENT,LAST MOD BY,LAST MOD DATE,LAST MOD TIME,FILE_PATH,FILENAME
10,10,24.906977,1.093023,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,0.823617,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,14,30.232558,0.697674,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,0.812980,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,21,50.627907,1.372093,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,0.955846,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,22,60.488372,1.511628,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,1.000000,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,29,0.860465,0.465116,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,55.092245,44100,Tolmomyias sulphurescens,1.000000,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30934,30934,0.000000,27.559200,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,Zimmerius-gracilipes-258669.wav,0,27.559200,44100,other,1.000000,NaN,31336.0,24000.0,0.0,217.078,alarce,10/18/2021,18:32:32,DELETE,Zimmerius-gracilipes-258669.wav
30935,30935,0.000000,27.559200,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,Zimmerius-gracilipes-258669.wav,0,27.559200,44100,other,1.000000,NaN,31337.0,24000.0,0.0,217.080,alarce,10/18/2021,18:32:32,DELETE,Zimmerius-gracilipes-258669.wav
30936,30936,0.000000,27.559200,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,Zimmerius-gracilipes-258669.wav,0,27.559200,44100,other,1.000000,NaN,31338.0,24000.0,0.0,217.079,alarce,10/18/2021,18:32:32,DELETE,Zimmerius-gracilipes-258669.wav
30937,30937,0.000000,27.559200,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,Zimmerius-gracilipes-258669.wav,0,27.559200,44100,other,1.000000,NaN,31339.0,24000.0,0.0,217.081,alarce,10/18/2021,18:32:32,DELETE,Zimmerius-gracilipes-258669.wav


In [23]:
def fix_folder_path(filepath):
    folder = "C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/"
    filename = filepath.split("/")[-1]#.replace(".WAV.", ".")
    return folder + filename
df["IN FILE"] = df["IN FILE"].apply(fix_folder_path)
df["IN FILE"][0]

C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0    C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...
0    C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...
Name: IN FILE, dtype: object

In [24]:
if (False):
    from opensoundscape.audio import Audio
    clip_duration = 3
    clip_overlap = 0
    final_clip = None
    clip_dir = 'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Cosmos_Data/Training_Cosmos'
    # classes = labels
    min_label_overlap = 0.1

    df_split_save = pd.DataFrame(columns=['file', 'start_time', 'end_time', 'SAMPLING RATE', 'MANUAL ID', "CONFIDENCE"])

    cnt = 0
    df = df.reset_index()
    for i, row in df.iterrows():
        try:
            audio = Audio.from_file(row['IN FILE'])
        except Exception:
            print(row['IN FILE'] + ' not found')
            continue
        file_name = row['IN FILE'].split('/')
        # print(file_name)
        clip_df = pd.DataFrame()
        offset = row["OFFSET"]
        duration = row["DURATION"]
        new_file = clip_dir + file_name[-1] + str(offset) + "s_" + str(offset+duration)+"s.wav"
        if (not os.path.exists(new_file)):
            clip_df = audio.split_and_save(
                clip_dir,
                prefix=file_name[-1],
                clip_duration=clip_duration,
                clip_overlap=clip_overlap,
                final_clip='remainder',
                dry_run=(os.path.exists(clip_dir + file_name[-1]))
            )
        else:
            tmp_df = pd.DataFrame()
            offset = row["OFFSET"]
            duration = row["DURATION"]
            tmp_df['file'] = new_file
            tmp_df['start_time'] = offset
            tmp_df['end_time'] = offset + duration
            if (clip_df.empty):
                clip_df = tmp_df
            else:
                clip_df = clip_df.append(tmp_df)

        clip_df['SAMPLING RATE'] = [44100] * clip_df.shape[0]
        clip_df['MANUAL ID'] = [row['MANUAL ID']] * clip_df.shape[0]
        clip_df.reset_index(inplace=True)
        df_split_save = df_split_save.append(clip_df, ignore_index=True)
        print(i, "out of", df.shape[0])

    print(df_split_save)


C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
!pip install pydub
from opensoundscape.audio import Audio
from pydub import AudioSegment
#AudioSegment.converter = "C:/Users/Siloux/Downloads/ffmpeg-master-latest-win64-gpl/ffmpeg-master-latest-win64-gpl/bin/ffmpeg.exe"
import os


clip_duration = 3
clip_overlap = 0
final_clip = None
clip_dir = 'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Training_Cosmos/'
# classes = labels
min_label_overlap = 0.1

df_split_save = pd.DataFrame(columns=['file', 'start_time', 'end_time', 'SAMPLING RATE', 'MANUAL ID', "CONFIDENCE"])

def convert_df_to_ops_df(row):
    offset = row["OFFSET"]
    duration = row["DURATION"]
    in_file_curr = row['IN FILE']
    file_name = in_file_curr.split('/')
    new_file = clip_dir + file_name[-1] + "_" + str(offset) + "s_" + str(offset+duration)+"s.wav"
    row['IN FILE'] = new_file
    
    if(not os.path.exists(new_file)):
        t1 = row["OFFSET"] * 1000
        # end time in milliseconds
        t2 = (row["OFFSET"] + 3) * 1000
        try:
            newAudio = AudioSegment.from_mp3(in_file_curr)
            newAudio = newAudio[t1:t2]
            new_file_object = newAudio.export(new_file, format="mp3")
            new_file_object.close()
        except Exception as e:
            print(e)
            row['IN FILE'] = "TO DELETE"
        #newAudio.close()
    
    return row
    

df_refractor = df.apply(convert_df_to_ops_df, axis=1)
df_refractor = df_refractor[df_refractor["IN FILE"] != "TO DELETE"]
df_refractor

C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedWriter name=4>
  return process_handler(cmd, _system_body)
C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)
C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedReader name=6>
  return process_handler(cmd, _system_body)
C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\pydub\audio_segment.py:796: ResourceWarning: unclosed file <_io.BufferedReader name='C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/XC148259 - Yellow-olive Flatbill - Tolmomyias sulphurescens confusus (based on locality and other XC cuts).wav'>
  return cls.from_file

[Errno 2] No such file or directory: 'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Training_Cosmos/XC148259 - Yellow-olive Flatbill - Tolmomyias sulphurescens confusus (based on locality and other XC cuts).wav_38.06976744186046s_38.76744186046512s.wav'
[Errno 2] No such file or directory: 'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Training_Cosmos/XC148259 - Yellow-olive Flatbill - Tolmomyias sulphurescens confusus (based on locality and other XC cuts).wav_38.97674418604651s_39.55813953488372s.wav'


C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\pydub\audio_segment.py:796: ResourceWarning: unclosed file <_io.BufferedReader name='C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/XC211446 - Southern Nightingale-Wren - Microcerculus marginatus marginatus (ssp assumed from location).wav'>
  return cls.from_file(file, 'mp3', parameters=parameters)
C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\pydub\audio_segment.py:796: ResourceWarning: unclosed file <_io.BufferedReader name='C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/XC211447 - Southern Nightingale-Wren - Microcerculus marginatus marginatus (ssp assumed from location).wav'>
  return cls.from_file(file, 'mp3', parameters=parameters)
C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\pydub\audio_segment.py:796: ResourceWarning: unclosed file 

[Errno 2] No such file or directory: 'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Training_Cosmos/XC211446 - Southern Nightingale-Wren - Microcerculus marginatus marginatus (ssp assumed from location).wav_0.5581395348837209s_5.116279069767442s.wav'
[Errno 2] No such file or directory: 'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Training_Cosmos/XC211446 - Southern Nightingale-Wren - Microcerculus marginatus marginatus (ssp assumed from location).wav_7.976744186046512s_8.581395348837209s.wav'
[Errno 2] No such file or directory: 'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Training_Cosmos/XC211446 - Southern Nightingale-Wren - Microcerculus marginatus marginatus (ssp assumed from location).wav_31.53488372093023s_32.906976744186046s.wav'
[Errno 2] No such file or directory: 'C:/Users/siloux/Desktop/E4E/pa

C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\pydub\audio_segment.py:796: ResourceWarning: unclosed file <_io.BufferedReader name='C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/XC211449 - Southern Nightingale-Wren - Microcerculus marginatus marginatus (ssp assumed from location).wav'>
  return cls.from_file(file, 'mp3', parameters=parameters)
C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\pydub\audio_segment.py:796: ResourceWarning: unclosed file <_io.BufferedReader name='C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/XC211450 - Southern Nightingale-Wren - Microcerculus marginatus marginatus (ssp assumed from location).wav'>
  return cls.from_file(file, 'mp3', parameters=parameters)
C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\pydub\audio_segment.py:796: ResourceWarning: unclosed file 

...

[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Vireolanius-leucotis-232203.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Vireolanius-leucotis-232203.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Vireolanius-leucotis-232203.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Vireolanius-leucotis-232203.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Vireolanius-leucot

[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Xiphorhynchus-elegans-271010.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Xiphorhynchus-elegans-271010.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Xiphorhynchus-elegans-271010.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Xiphorhynchus-elegans-271010.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Xiphorhynchus-

[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Zenaida-auriculata-77090.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Zenaida-auriculata-77090.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Zenaida-auriculata-77090.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Zenaida-auriculata-77090.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Zenaida-auriculata-77090.wav'


[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Zimmerius-gracilipes-258669.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Zimmerius-gracilipes-258669.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Zimmerius-gracilipes-258669.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Zimmerius-gracilipes-258669.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Zimmerius-gracilip

,Unnamed: 0,OFFSET,DURATION,FOLDER,IN FILE,CHANNEL,CLIP LENGTH,SAMPLE RATE,MANUAL ID,CONFIDENCE,FILEPATH,Unnamed: 0.1,MAX FREQ,MIN FREQ,TIME_SPENT,LAST MOD BY,LAST MOD DATE,LAST MOD TIME,FILE_PATH,FILENAME
10,10,24.906977,1.093023,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,0.823617,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,14,30.232558,0.697674,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,0.812980,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,21,50.627907,1.372093,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,0.955846,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,22,60.488372,1.511628,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,1.000000,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,29,0.860465,0.465116,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0,55.092245,44100,Tolmomyias sulphurescens,1.000000,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30913,30913,0.000000,26.906100,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0,26.906100,44100,other,1.000000,NaN,31315.0,24000.0,0.0,145.328,alarce,10/18/2021,18:27:07,NaN,Zimmerius-cinereicapilla-258673.wav
30914,30914,0.000000,12.199200,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0,12.199200,44100,other,1.000000,NaN,31316.0,24000.0,0.0,77.548,alarce,10/18/2021,18:28:44,NaN,Zimmerius-gracilipes-258668.wav
30915,30915,0.000000,12.199200,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0,12.199200,44100,other,1.000000,NaN,31317.0,24000.0,0.0,77.549,alarce,10/18/2021,18:28:44,NaN,Zimmerius-gracilipes-258668.wav
30916,30916,0.000000,12.199200,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0,12.199200,44100,other,1.000000,NaN,31318.0,24000.0,0.0,77.550,alarce,10/18/2021,18:28:44,NaN,Zimmerius-gracilipes-258668.wav


In [26]:
df_refractor

C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,OFFSET,DURATION,FOLDER,IN FILE,CHANNEL,CLIP LENGTH,SAMPLE RATE,MANUAL ID,CONFIDENCE,FILEPATH,Unnamed: 0.1,MAX FREQ,MIN FREQ,TIME_SPENT,LAST MOD BY,LAST MOD DATE,LAST MOD TIME,FILE_PATH,FILENAME
10,10,24.906977,1.093023,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,0.823617,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,14,30.232558,0.697674,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,0.812980,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,21,50.627907,1.372093,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,0.955846,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,22,60.488372,1.511628,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,1.000000,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,29,0.860465,0.465116,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0,55.092245,44100,Tolmomyias sulphurescens,1.000000,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30913,30913,0.000000,26.906100,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0,26.906100,44100,other,1.000000,NaN,31315.0,24000.0,0.0,145.328,alarce,10/18/2021,18:27:07,NaN,Zimmerius-cinereicapilla-258673.wav
30914,30914,0.000000,12.199200,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0,12.199200,44100,other,1.000000,NaN,31316.0,24000.0,0.0,77.548,alarce,10/18/2021,18:28:44,NaN,Zimmerius-gracilipes-258668.wav
30915,30915,0.000000,12.199200,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0,12.199200,44100,other,1.000000,NaN,31317.0,24000.0,0.0,77.549,alarce,10/18/2021,18:28:44,NaN,Zimmerius-gracilipes-258668.wav
30916,30916,0.000000,12.199200,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0,12.199200,44100,other,1.000000,NaN,31318.0,24000.0,0.0,77.550,alarce,10/18/2021,18:28:44,NaN,Zimmerius-gracilipes-258668.wav


In [27]:
df_split_save = df_refractor[["IN FILE", "OFFSET", "DURATION", "SAMPLE RATE", "MANUAL ID"]]
df_split_save["end_time"] = df_split_save["OFFSET"] + df_split_save["DURATION"]
df_split_save = df_split_save.rename(columns={"IN FILE": "file", "OFFSET": "start_time"})
df_split_save = df_split_save[['file', 'start_time', 'end_time', 'SAMPLE RATE', 'MANUAL ID']]

C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [28]:
df_split_save

,file,start_time,end_time,SAMPLE RATE,MANUAL ID
10,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,24.906977,26.000000,44100,Microcerculus marginatus
14,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,30.232558,30.930233,44100,Microcerculus marginatus
21,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,50.627907,52.000000,44100,Microcerculus marginatus
22,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,60.488372,62.000000,44100,Microcerculus marginatus
29,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.860465,1.325581,44100,Tolmomyias sulphurescens
...,...,...,...,...,...
30913,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.000000,26.906100,44100,other
30914,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.000000,12.199200,44100,other
30915,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.000000,12.199200,44100,other
30916,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.000000,12.199200,44100,other


In [29]:
train_name

C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'./automated_cosmos_tweety_to_file_TRAINING.csv'

In [30]:
df_split_save.to_csv(train_name)
df_split_save

,file,start_time,end_time,SAMPLE RATE,MANUAL ID
10,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,24.906977,26.000000,44100,Microcerculus marginatus
14,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,30.232558,30.930233,44100,Microcerculus marginatus
21,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,50.627907,52.000000,44100,Microcerculus marginatus
22,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,60.488372,62.000000,44100,Microcerculus marginatus
29,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.860465,1.325581,44100,Tolmomyias sulphurescens
...,...,...,...,...,...
30913,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.000000,26.906100,44100,other
30914,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.000000,12.199200,44100,other
30915,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.000000,12.199200,44100,other
30916,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.000000,12.199200,44100,other


In [31]:
pd.set_option('display.width', 10000)
pd.set_option('display.expand_frame_repr', False)
df["merge file"] = df["IN FILE"].apply(lambda x: x.split("/")[-1])
#df_split_save['CONFIDENCE'] = df_split_save['CONFIDENCE'].fillna(-1)
#for i in range(df.shape[0]):
##    if (i % 100 == 0):
#        print(i)
#    row = df.iloc[i]
#    rows_to_change = (df_split_save["file"].str.contains(row["merge file"])) & (df_split_save["start_time"] == (row["Start (s)"])) & (df_split_save["end_time"] == (row["End (s)"]))
#    df_split_save.loc[rows_to_change, "CONFIDENCE"]= row["Confidence"]
df_split_save

C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,file,start_time,end_time,SAMPLE RATE,MANUAL ID
10,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,24.906977,26.000000,44100,Microcerculus marginatus
14,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,30.232558,30.930233,44100,Microcerculus marginatus
21,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,50.627907,52.000000,44100,Microcerculus marginatus
22,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,60.488372,62.000000,44100,Microcerculus marginatus
29,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.860465,1.325581,44100,Tolmomyias sulphurescens
...,...,...,...,...,...
30913,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.000000,26.906100,44100,other
30914,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.000000,12.199200,44100,other
30915,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.000000,12.199200,44100,other
30916,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.000000,12.199200,44100,other


In [32]:
df_split_save.drop_duplicates()

C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,file,start_time,end_time,SAMPLE RATE,MANUAL ID
10,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,24.906977,26.000000,44100,Microcerculus marginatus
14,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,30.232558,30.930233,44100,Microcerculus marginatus
21,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,50.627907,52.000000,44100,Microcerculus marginatus
22,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,60.488372,62.000000,44100,Microcerculus marginatus
29,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.860465,1.325581,44100,Tolmomyias sulphurescens
...,...,...,...,...,...
30742,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.000000,24.792000,44100,other
30867,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.000000,5.120000,44100,other
30879,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.000000,36.702000,44100,other
30909,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.000000,26.906100,44100,other


In [33]:
import os
from os import listdir
from os.path import isfile, join

test_path = "C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Testing_Cosmos/"
test_files = [f for f in listdir(test_path) if isfile(join(test_path, f))]
train_path = "C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Training_Cosmos/"
train_files = [f for f in listdir(train_path) if isfile(join(train_path, f))]
overlapping_files = []
for item in train_files:
    if (item in test_files):
        path = train_path + "/" + item
        os.remove(path)
        overlapping_files.append(path)

df_split_save2 = df_split_save[~df_split_save["file"].isin(overlapping_files)]
df_split_save2.to_csv(train_name)       
print(len(overlapping_files))
df_split_save2

C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0


,file,start_time,end_time,SAMPLE RATE,MANUAL ID
10,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,24.906977,26.000000,44100,Microcerculus marginatus
14,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,30.232558,30.930233,44100,Microcerculus marginatus
21,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,50.627907,52.000000,44100,Microcerculus marginatus
22,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,60.488372,62.000000,44100,Microcerculus marginatus
29,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.860465,1.325581,44100,Tolmomyias sulphurescens
...,...,...,...,...,...
30913,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.000000,26.906100,44100,other
30914,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.000000,12.199200,44100,other
30915,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.000000,12.199200,44100,other
30916,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.000000,12.199200,44100,other


In [34]:
test_path2 = "C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Testing_Cosmos/"
test_files2 = [f for f in listdir(test_path2) if isfile(join(test_path2, f))]
train_path2 =  "C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Training_Cosmos/"
train_files2 = [f for f in listdir(train_path2) if isfile(join(train_path2, f))]
overlapping_files2 = []
for item in train_files2:
    if (item in test_files2):
        overlapping_files2.append(item)
overlapping_files2

C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[]

In [35]:
df_split_save2 = df_split_save[~df_split_save["file"].isin(overlapping_files)]
df_split_save2.to_csv(train_name)       
print(len(overlapping_files))

0


In [36]:
#df_split_save2.loc[df_split_save2["CONFIDENCE"] == -1, "CONFIDENCE"] = 0
#df_split_save2 = df_split_save2.loc[df_split_save2["CONFIDENCE"] > 0.5]
df_split_save2.to_csv(train_name) 

In [37]:
df_split_save2

,file,start_time,end_time,SAMPLE RATE,MANUAL ID
10,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,24.906977,26.000000,44100,Microcerculus marginatus
14,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,30.232558,30.930233,44100,Microcerculus marginatus
21,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,50.627907,52.000000,44100,Microcerculus marginatus
22,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,60.488372,62.000000,44100,Microcerculus marginatus
29,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.860465,1.325581,44100,Tolmomyias sulphurescens
...,...,...,...,...,...
30913,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.000000,26.906100,44100,other
30914,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.000000,12.199200,44100,other
30915,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.000000,12.199200,44100,other
30916,C:/Users/siloux/Desktop/E4E/passive-acoustic-b...,0.000000,12.199200,44100,other


In [38]:
np.unique(df_split_save2["MANUAL ID"])

C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['Celeus grammicus', 'Microcerculus marginatus',
       'Myioborus miniatus', 'Ramphastos tucanus', 'Thraupis episcopus',
       'Tolmomyias sulphurescens', 'Trogon viridis', 'Turdus leucomelas',
       'Xiphorhynchus guttatus', 'Zonotrichia capensis', 'other'],
      dtype=object)